# Chapter 11. Training Deep Neural Networks


### The Vanishing/Exploding Gradients Problems


### Glorot and He Initialization

- By default, Keras uses Glorot initialization with a uniform distribution. When you create a layer, you can switch to He initialization by setting kernel_initializer="he_uniform" or kernel_initializer="he_normal" like this:


| Initialization | Activation functions                     | σ² (Normal) |
| -------------- | ---------------------------------------- | ----------- |
| Glorot         | None, tanh, sigmoid, softmax             | 1 / fan_avg |
| He             | ReLU, Leaky ReLU, ELU, GELU, Swish, Mish | 2 / fan_in  |
| LeCun          | SELU                                     | 1 / fan_in  |


In [1]:
import tensorflow as tf

dense = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")

- Alternatively, you can obtain any of the initializations listed in **Table** and more using the `VarianceScaling` initializer. For example, if you want **He initialization** with a uniform distribution and based on _fan_avg_ (rather than _fan_in_), you can use the following code:


In [2]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg", distribution="uniform")
dense = tf.keras.layers.Dense(
    50,
    activation="sigmoid",
    kernel_initializer=he_avg_init #Here it is...
)

### Better Activation Functions


#### Leaky ReLU

- Keras includes the classes LeakyReLU and PReLU in the tf.keras.layers package. Just like for other ReLU variants, you should use He initialization with these. For example:


In [3]:
leaky_Relu = tf.keras.layers.LeakyReLU(negative_slope=0.2)

dense = tf.keras.layers.Dense(50, activation=leaky_Relu, kernel_initializer="he_normal")

- If you prefer, you can also use LeakyReLU as a separate layer in your model; it makes no difference for training and predictions:


In [4]:
# model = tf.keras.models.Sequential([
#     [...], # more layers
#     tf.keras.layers.Dense(50, kernel_initializer="he_normal"), # no activation
#     tf.keras.layers.LeakyReLU(alpha=0.2), # activation as a separate layer
#     [...] # more layers
# ])

#### ELU and SELU


In [5]:
# ELU
dense = tf.keras.layers.Dense(50, activation="elu", kernel_initializer="he_normal")

In [6]:
#SELU
dense = tf.keras.layers.Dense(50, activation="selu", kernel_initializer="lecun_normal")

#### Batch Normalization

- Implementing batch normalization with Keras


- As with most things with Keras, implementing batch normalization is straightforward and intuitive. Just add a BatchNormalization layer before or after each hidden layer’s activation function. You may also add a BN layer as the first layer in your model, but a plain Normalization layer generally performs just as well in this location.


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input([28, 28]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 271,346 (1.04 MB)

 Trainable params: 268,978 (1.03 MB)

 Non-trainable params: 2,368 (9.25 KB)

- Let’s look at the parameters of the first BN layer. Two are trainable (by backpropagation), and two are not:


In [9]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('gamma', True),
 ('beta', True),
 ('moving_mean', False),
 ('moving_variance', False)]

- The authors of the BN paper argued in favor of adding the BN layers before the activation functions, rather than after (as we just did). There is some debate about this, as which is preferable seems to depend on the task—you can experiment with this too to see which option works best on your dataset. To add the BN layers before the activation function, you must remove the activation functions from the hidden layers and add them as separate layers after the BN layers. Moreover, since a batch normalization layer includes one offset parameter per input, you can remove the bias term from the previous layer by passing use_bias=False when creating it. Lastly, you can usually drop the first BN layer to avoid sandwiching the first hidden layer between two BN layers. The updated code looks like this:


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Input([28, 28]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

#### Gradient Clipping


In [11]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)  # Gradyan bileşenlerini sınırla
model.compile(optimizer=optimizer, loss="mse")

# veya gradyan normunu sınırla
optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(optimizer=optimizer, loss="mse")

### Reusing Pretrained Layers


- If you have trained model, you can use it for another related model. It called Transfer Learning.


#### Transfer Learning with Keras


- First, you need to load model A and create a new model based on that model’s layers. You decide to reuse all the layers except for the output layer:


In [ ]:
[...] # Assuming model A was already trained and saved to "my_model_A"
model_A = tf.keras.models.load_model("my_model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

- Note that model_A and model_B_on_A now share some layers. When you train model_B_on_A, it will also affect model_A. If you want to avoid that, you need to clone model_A before you reuse its layers. To do this, you clone model A’s architecture with clone_model(), then copy its weights:


In [ ]:
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weigths(model_A.get_weights())

- Since the new output layer was initialized randomly it will make large errors (at least during the first few epochs), so there will be large error gradients that may wreck the reused weights. To avoid this, one approach is to freeze the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights. To do this, set every layer’s trainable attribute to False and compile the model:


In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
    
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

* Now you can train the model for a few epochs, then unfreeze the reused layers (which requires compiling the model again) and continue training to fine-tune the reused layers for task B. After unfreezing the reused layers, it is usually a good idea to reduce the learning rate, once again to avoid damaging the reused weights.

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

model_B_on_A.evaluate(X_test_B, y_test_B)

#### Unsupervised Pretraining

* Note that in the early days of deep learning it was difficult to train deep models, so people would use a technique called greedy layer-wise pretraining (depicted in Figure 11-6). They would first train an unsupervised model with a single layer, typically an RBM, then they would freeze that layer and add another one on top of it, then train the model again (effectively just training the new layer), then freeze the new layer and add another layer on top of it, train the model again, and so on. Nowadays, things are much simpler: people generally train the full unsupervised model in one shot and use autoencoders or GANs rather than RBMs.

### Faster Optimizers

* Another huge speed boost comes from using a faster optimizer than the regular gradient descent optimizer. In this section we will present the most popular optimization algorithms: momentum, Nesterov accelerated gradient, AdaGrad, RMSProp, and finally Adam and its variants.

#### Momentum

* Imagine a bowling ball rolling down a gentle slope on a smooth surface: it will start out slowly, but it will quickly pick up momentum until it eventually reaches terminal velocity (if there is some friction or air resistance). This is the core idea behind momentum optimization, In contrast, regular gradient descent will take small steps when the slope is gentle and big steps when the slope is steep, but it will never pick up speed. As a result, regular gradient descent is generally much slower to reach the minimum than momentum optimization.

In [20]:
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.001,
    momentum=0.9 #Critical Point
)
model.compile(optimizer=optimizer, loss="mse")

#### Nesterov Accelerated Gradient

* Nesterov momentum optimization, measures the gradient of the cost function not at the local position θ but slightly ahead in the direction of the momentum, at θ + βm

* This small tweak works because in general the momentum vector will be pointing in the right direction (i.e., toward the optimum), so it will be slightly more accurate to use the gradient measured a bit farther in that direction rather than the gradient at the original position.

* To use NAG, simply set nesterov=True when creating the SGD optimizer:

In [21]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Tek fark: nesterov=True parametresinin eklenmesidir.

#### Ada Grad

* Consider the elongated bowl problem again: gradient descent starts by quickly going down the steepest slope, which does not point straight toward the global optimum, then it very slowly goes down to the bottom of the valley. It would be nice if the algorithm could correct its direction earlier to point a bit more toward the global optimum. The AdaGrad algorithm⁠ achieves this correction by scaling down the gradient vector along the steepest dimensions.

#### RMProp

* As we’ve seen, AdaGrad runs the risk of slowing down a bit too fast and never converging to the global optimum. The RMSProp algorithm⁠ fixes this by accumulating only the gradients from the most recent iterations, as opposed to all the gradients since the beginning of training. It does so by using exponential decay in the first step.

* As you might expect, Keras has an RMSprop optimizer:

In [22]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

* Except on very simple problems, this optimizer almost always performs much better than AdaGrad. In fact, it was the preferred optimization algorithm of many researchers until Adam optimization came around.

#### Adam 🚀 (Most Used One)

* Adam,⁠ which stands for adaptive moment estimation, combines the ideas of momentum optimization and RMSProp: just like momentum optimization, it keeps track of an exponentially decaying average of past gradients; and just like RMSProp, it keeps track of an exponentially decaying average of past squared gradients (see Equation 11-9). These are estimations of the mean and (uncentered) variance of the gradients. The mean is often called the first moment while the variance is often called the second moment, hence the name of the algorithm.

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# β1 ​(momentum katsayısı) genellikle 0.9
# β2 ​(RMSProp’un ölçekleme faktörü) genellikle 0.999

#### AdaMax

* In practice, this can make AdaMax more stable than Adam, but it really depends on the dataset, and in general Adam performs better. So, this is just one more optimizer you can try if you experience problems with Adam on some task.

#### Nadam

* Nadam optimization is Adam optimization plus the Nesterov trick, so it will often converge slightly faster than Adam. In his report introducing this technique,⁠ the researcher Timothy Dozat compares many different optimizers on various tasks and finds that Nadam generally outperforms Adam but is sometimes outperformed by RMSProp.

#### AdamW

* AdamW⁠ is a variant of Adam that integrates a regularization technique called weight decay. Weight decay reduces the size of the model’s weights at each training iteration by multiplying them by a decay factor such as 0.99. This may remind you of ℓ regularization, which also aims to keep the weights small, and indeed it can be shown mathematically that ℓ regularization is equivalent to weight decay when using SGD. However, when using Adam or its variants, ℓ regularization and weight decay are not equivalent: in practice, combining Adam with ℓ regularization results in models that often don’t generalize as well as those produced by SGD. AdamW fixes this issue by properly combining Adam with weight decay.

### Learning Rates Scheduling

* As discussed in Chapter 10, you can find a good learning rate by training the model for a few hundred iterations, exponentially increasing the learning rate from a very small value to a very large value, and then looking at the learning curve and picking a learning rate slightly lower than the one at which the learning curve starts shooting back up. You can then reinitialize your model and train it with that learning rate.

* But you can do better than a constant learning rate: if you start with a large learning rate and then reduce it once training stops making fast progress, you can reach a good solution faster than with the optimal constant learning rate. There are many different strategies to reduce the learning rate during training. It can also be beneficial to start with a low learning rate, increase it, then drop it again. These strategies are called learning schedules (I briefly introduced this concept in Chapter 4). These are the most commonly used learning schedules:

- Power scheduling
    * Set the learning rate to a function of the iteration number t: η(t) = η / (1 + t/s) . The initial learning rate η , the power c (typically set to 1), and the steps s are hyperparameters. The learning rate drops at each step. After s steps, the learning rate is down to η / 2. After s more steps it is down to η / 3, then it goes down to η / 4, then η / 5, and so on. As you can see, this schedule first drops quickly, then more and more slowly. Of course, power scheduling requires tuning η and s (and possibly c).

- Exponential scheduling
    * Set the learning rate to η(t) = η 0.1 . The learning rate will gradually drop by a factor of 10 every s steps. While power scheduling reduces the learning rate more and more slowly, exponential scheduling keeps slashing it by a factor of 10 every s steps.

- Piecewise constant scheduling
    * Use a constant learning rate for a number of epochs (e.g., η = 0.1 for 5 epochs), then a smaller learning rate for another number of epochs (e.g., η = 0.001 for 50 epochs), and so on. Although this solution can work very well, it requires fiddling around to figure out the right sequence of learning rates and how long to use each of them.

- Performance scheduling
    * Measure the validation error every N steps (just like for early stopping), and reduce the learning rate by a factor of λ when the error stops dropping.

- 1cycle scheduling
    * 1cycle was introduced in a 2018 paper by Leslie Smith.⁠ Contrary to the other approaches, it starts by increasing the initial learning rate η , growing linearly up to η halfway through training. Then it decreases the learning rate linearly down to η again during the second half of training, finishing the last few epochs by dropping the rate down by several orders of magnitude (still linearly). The maximum learning rate η is chosen using the same approach we used to find the optimal learning rate, and the initial learning rate η is usually 10 times lower. When using a momentum, we start with a high momentum first (e.g., 0.95), then drop it down to a lower momentum during the first half of training (e.g., down to 0.85, linearly), and then bring it back up to the maximum value (e.g., 0.95) during the second half of training, finishing the last few epochs with that maximum value. Smith did many experiments showing that this approach was often able to speed up training considerably and reach better performance. For example, on the popular CIFAR10 image dataset, this approach reached 91.9% validation accuracy in just 100 epochs, compared to 90.3% accuracy in 800 epochs through a standard approach (with the same neural network architecture). This feat was dubbed superconvergence.

* Implementing power scheduling in Keras is the easiest option—just set the decay hyperparameter when creating an optimizer:

In [28]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

# Burada decay =  1e−4, yani her adımda öğrenme oranı hafifçe azalır.

* Exponential scheduling and piecewise scheduling are quite simple too. You first need to define a function that takes the current epoch and returns the learning rate. For example, let’s implement exponential scheduling:

In [29]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

# Epoch arttıkça öğrenme oranı küçülür, böylece model yavaşça en iyi hale gelir.

* If you do not want to hardcode η and s, you can create a function that returns a configured function:

In [ ]:
def exponential_decay(lr_0, s):
    def exponential_decay_fn(epoch):
        return lr_0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr_0=0.01, s=20)

# lr_0 → Başlangıç öğrenme oranı
# s → Öğrenme oranı düşürme süresi


* Next, create a LearningRateScheduler callback, giving it the schedule function, and pass this callback to the fit() method:

In [ ]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train, y_train, [...], callbacks=[lr_scheduler])

* The schedule function can optionally take the current learning rate as a second argument. For example, the following schedule function multiplies the previous learning rate by 0.1 , which results in the same exponential decay (except the decay now starts at the beginning of epoch 0 instead of 1):

In [33]:
def exponential_decay_fn(epoch, lr):
    return lr * 0.1 ** (1 / 20)

* For piecewise constant scheduling, you can use a schedule function like the following one (as earlier, you can define a more general function if you want; see the “Piecewise Constant Scheduling” section of the notebook for an example), then create a LearningRateScheduler callback with this function and pass it to the fit() method, just like for exponential scheduling:

In [35]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

* For performance scheduling, use the ReduceLROnPlateau callback. For example, if you pass the following callback to the fit() method, it will multiply the learning rate by 0.5 whenever the best validation loss does not improve for five consecutive epochs (other options are available; please check the documentation for more details):

In [ ]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model.fit(X_train, y_train, [...], callbacks=[lr_scheduler])

* Lastly, Keras offers an alternative way to implement learning rate scheduling: you can define a scheduled learning rate using one of the classes available in tf.keras.opti⁠ mizers.schedules, then pass it to any optimizer. This approach updates the learning rate at each step rather than at each epoch. For example, here is how to implement the same exponential schedule as the exponential_decay_fn() function we defined earlier:

In [ ]:
import math

batch_size = 32
n_epochs = 25
n_steps = n_epochs * math.ceil(len(X_train) / batch_size)

scheduled_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, decay_steps=n_steps, decay_rate=0.1
)

optimizer = tf.keras.optimizers.SGD(learning_rate=scheduled_learning_rate)

### Avoiding Overfitting Through Regularization

* We already implemented one of the best regularization techniques in Chapter 10: early stopping. Moreover, even though batch normalization was designed to solve the unstable gradients problems, it also acts like a pretty good regularizer. In this section we will examine other popular regularization techniques for neural networks: ℓ and ℓ regularization, dropout, and maxnorm regularization.

#### ℓ1 and ℓ2 Regularization

* Just like you did in Chapter 4 for simple linear models, you can use ℓ regularization to constrain a neural network’s connection weights, and/or ℓ regularization if you want a sparse model (with many weights equal to 0). Here is how to apply ℓ regularization to a Keras layer’s connection weights, using a regularization factor of 0.01:

In [37]:
layer = tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)
)

# Eğer L1 regularization istenirse:
# kernel_regularizer=tf.keras.regularizers.l1(0.01)

# Eğer hem L1 hem de L2 regularization istenirse:
# kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)

* Since you will typically want to apply the same regularizer to all layers in your network, as well as using the same activation function and the same initialization strategy in all hidden layers, you may find yourself repeating the same arguments. This makes the code ugly and error-prone. To avoid this, you can try refactoring your code to use loops. Another option is to use Python’s functools.partial() function, which lets you create a thin wrapper for any callable, with some default argument values:

In [38]:
from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01)
)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    tf.keras.layers.Dense(10, activation="softmax")
])


c:\Users\MRE\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Dropout

* It is a fairly simple algorithm: at every training step, every neuron (including the input neurons, but always excluding the output neurons) has a probability p of being temporarily “dropped out”, meaning it will be entirely ignored during this training step, but it may be active during the next step.

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
    kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
    kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])
[...] # compile and train the model

#### Monte Carlo (MC) Dropout

* It is the full implementation of MC dropout, boosting the dropout model we trained earlier without retraining it:

In [ ]:
import numpy as np
y_probas = np.stack([model(X_test, training=True) 
                     for sample in range(100)])

y_proba = y_probas.mean(axis=0)

* We just make 100 predictions over the test set, and we compute their average.

#### Max-Norm Regularization

* To implement max-norm regularization in Keras, set the kernel_constraint argument of each hidden layer to a max_norm() constraint with the appropriate max value, like this:

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.constraints import max_norm

dense = Dense(100, activation="relu",
              kernel_initializer="he_normal",
              kernel_constraint=max_norm(1.))